In [ ]:
from glob import glob
import json
import pandas as pd
from itertools import product
import numpy as np
from scipy.optimize import least_squares
from functools import partial

from utils import make_countmatrix, sort_schools, sigmoid, logit, compute_p, compute_mu, to_np, cost

import matplotlib.pyplot as plt
import json
from glob import glob
import re
from IPython.display import display

In [ ]:
prefix = 'pilot_general_multiple'
prefix = 'second_pilot'
prefix = 'experiment'
# prefix = 'pilot_personal_multiple'
data_dir = '../data/processed/'
vps = set(int(re.findall("\d+", l)[0]) for l in glob(data_dir+prefix+"*"))
print(vps)

In [ ]:
def parabola(x, w):
    return w[0] + w[1] * x + w[2] * x**2

def pcost(data, w):
    return sum((parabola(d[0], w) - d[1])**2 for d in data)

In [ ]:
# points = [(stimulus_wr[k], v) for k, v in score.items()]
# w = least_squares(partial(pcost, points), [0, 0, 0]).x
# X = np.linspace(-1, 1)
# points = [(x, parabola(x, (0.1, -0.1, 3))) for x in X]
# w = least_squares(partial(pcost, points), [0, 0, 0]).x
# w
# y = np.array(p

In [ ]:
# X = np.array([p[1] for p in points])
# A = np.vstack([np.ones(len(X)), X, X**2]).T
# w = np.linalg.lstsq(A, [p[1] for p in points])[0]
# w

In [ ]:
with open(data_dir + 'stimulus_wr.json') as fh:
    stimulus_wr = json.load(fh)

g2c = {
    'low': 'red',
    'medium': 'blue',
    'high': 'green'
}

avg_wr = {f: stimulus_wr[f'avgimg_wr_{f}.png'] for f in g2c.keys()}

for vp in vps:
    for g in ['low', 'medium', 'high']:
        plt.vlines(avg_wr[g], -0.5, 0.5, color=g2c[g])
        name = data_dir + f'{prefix}_vp-{vp}_g-{g}.pkl'
        data = pd.read_pickle(name)
        display(data)
        print((data.win1 + data.win2).mean())

        p = dict((data.win1 / (data.win1 + data.win2)))
        p = dict(compute_p(data))
        p = dict(filter(lambda x: ~np.isnan(x[1]), p.items()))

        school2idx = {y: x for x, y in enumerate(set(y for x in data.index for y in x))}
        A = make_countmatrix(data, school2idx)
        schools_sorted = sort_schools(A, school2idx)
        # rev_schools_sorted = list(reversed(schools_sorted))

        # optimized_mu = least_squares(cost, to_np(mu_1)).x
        N = len(schools_sorted)
        f = partial(cost, schools_sorted, p)
        optimized_mu = least_squares(f, np.zeros(N)).x
        score = {city: optimized_mu[i] for i, city in enumerate(schools_sorted[:-1])}
        score[schools_sorted[-1]] = 0
        
        points = [(stimulus_wr[k], v) for k, v in score.items()]
        print(points)
        # w = least_squares(partial(pcost, points), [0, 0, 0]).x
        X = np.array([p[0] for p in points])
        A = np.vstack([np.ones(len(X)), X, X**2]).T
        w = np.linalg.lstsq(A, [p[1] for p in points])[0]
        x = np.linspace(0.43, 0.49)
        y = parabola(x, w)
        plt.plot(x, y, color=g2c[g])

        # plt.plot([stimulus_wr[key] for key in score], score.values(), 'x')
        plt.title(name)
        for key, value in score.items():
            label = key
            label = None
            plt.plot(stimulus_wr[key], value, 'x', label=label, color=g2c[g])
        plt.legend()
    plt.show()

In [ ]:
display(data)
print((data.win1 + data.win2).mean())

In [ ]:
p

In [ ]:
for g in ['low', 'medium', 'high']:
    plt.vlines(avg_wr[g], -1.5, 1.0, ls='--', color=g2c[g], alpha=0.5)
    name = data_dir + f'pooled_{prefix}_g-{g}.pkl'
    data = pd.read_pickle(name)
    display(data)
    print((data.win1 + data.win2).mean())

    p = dict((data.win1 / (data.win1 + data.win2)))
    p = dict(compute_p(data))
    p = dict(filter(lambda x: ~np.isnan(x[1]), p.items()))

    school2idx = {y: x for x, y in enumerate(set(y for x in data.index for y in x))}
    A = make_countmatrix(data, school2idx)
    schools_sorted = sort_schools(A, school2idx)
    # rev_schools_sorted = list(reversed(schools_sorted))

    # optimized_mu = least_squares(cost, to_np(mu_1)).x
    N = len(schools_sorted)
    f = partial(cost, schools_sorted, p)
    optimized_mu = least_squares(f, np.zeros(N)).x
    score = {city: optimized_mu[i] for i, city in enumerate(schools_sorted[:-1])}
    score[schools_sorted[-1]] = 0

    points = [(stimulus_wr[k], v) for k, v in score.items()]
    print(points)
    # w = least_squares(partial(pcost, points), [0, 0, 0]).x
    X = np.array([p[0] for p in points])
    A = np.vstack([np.ones(len(X)), X, X**2]).T
    w = np.linalg.lstsq(A, [p[1] for p in points])[0]
    x = np.linspace(0.43, 0.49)
    y = parabola(x, w)
    plt.plot(x, y, color=g2c[g])

    # plt.plot([stimulus_wr[key] for key in score], score.values(), 'x')
    for key, value in score.items():
        label = key
        label = None
        plt.plot(stimulus_wr[key], value, 'x', label=label, color=g2c[g])
# plt.legend()
plt.title('pooled')
plt.savefig('pooled_3vp_pilot.pdf')
plt.show()

In [ ]:
data.to_excel('pooled_data.xlsx')

In [ ]:
import pymc3 as pm
from scipy.special import ndtri, ndtr
    
p = data.win1 / (data.win1 + data.win2)
p

In [ ]:
imgs = [-100, -66, -33, 0, 33, 66, 100]
probs = {}
for i in range(len(imgs)):
    for j in range(i + 1, len(imgs)):
        left = f'high_({imgs[i]}).png' if imgs[i] != 0 else 'avgimg_wr_high.png'
        right = f'high_({imgs[j]}).png' if imgs[j] != 0 else 'avgimg_wr_high.png'
        try:
            probs[(imgs[i], imgs[j])] = p.loc[(left, right)]
        except KeyError:
            probs[(imgs[i], imgs[j])] = p.loc[(right, left)]
probs

In [ ]:
n_imgs = 7
with pm.Model() as thurstone:
    #u_i = pm.Normal("u_i", mu=0, sigma=1, shape=n_img)
    u = [pm.Normal(f"u_{i}", mu=0, sigma=1) for i in range(n_imgs)]
    for i in range(len(imgs)):
        for j in range(i + 1, len(imgs)):
            pass
#             pm.Binomial()
#             ndtr(u[i] - u[j])
#             probs[(imgs[i], imgs[j])] 